In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score,recall_score,confusion_matrix
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
dataSet = pd.read_csv('horse.csv')
dataSet.head()

,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,...,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,outcome,surgical_lesion,lesion_1,lesion_2,lesion_3,cp_data
0,no,adult,530101,38.5,66.0,28.0,cool,reduced,NaN,more_3_sec,...,45.0,8.4,NaN,NaN,died,no,11300,0,0,no
1,yes,adult,534817,39.2,88.0,20.0,NaN,NaN,pale_cyanotic,less_3_sec,...,50.0,85.0,cloudy,2.0,euthanized,no,2208,0,0,no
2,no,adult,530334,38.3,40.0,24.0,normal,normal,pale_pink,less_3_sec,...,33.0,6.7,NaN,NaN,lived,no,0,0,0,yes
3,yes,young,5290409,39.1,164.0,84.0,cold,normal,dark_cyanotic,more_3_sec,...,48.0,7.2,serosanguious,5.3,died,yes,2208,0,0,yes
4,no,adult,530255,37.3,104.0,35.0,NaN,NaN,dark_cyanotic,more_3_sec,...,74.0,7.4,NaN,NaN,died,no,4300,0,0,no


In [3]:
#droppping reduntant label
dataSet.drop('cp_data',axis=1,inplace=True)

In [4]:
#dataSet.head()
target=dataSet['outcome']
dataSet.drop('outcome',axis=1,inplace=True)


In [5]:
#encoding catagorical values 
dataSet = pd.get_dummies(dataSet)
dataSet.head()

,hospital_number,rectal_temp,pulse,respiratory_rate,nasogastric_reflux_ph,packed_cell_volume,total_protein,abdomo_protein,lesion_1,lesion_2,...,abdomen_distend_large,abdomen_distend_small,abdomen_firm,abdomen_normal,abdomen_other,abdomo_appearance_clear,abdomo_appearance_cloudy,abdomo_appearance_serosanguious,surgical_lesion_no,surgical_lesion_yes
0,530101,38.5,66.0,28.0,NaN,45.0,8.4,NaN,11300,0,...,1,0,0,0,0,0,0,0,1,0
1,534817,39.2,88.0,20.0,NaN,50.0,85.0,2.0,2208,0,...,0,0,0,0,1,0,1,0,1,0
2,530334,38.3,40.0,24.0,NaN,33.0,6.7,NaN,0,0,...,0,0,0,1,0,0,0,0,1,0
3,5290409,39.1,164.0,84.0,5.0,48.0,7.2,5.3,2208,0,...,0,0,0,0,0,0,0,1,0,1
4,530255,37.3,104.0,35.0,NaN,74.0,7.4,NaN,4300,0,...,0,0,0,0,0,0,0,0,1,0


In [6]:
dataSet.fillna(method="bfill",inplace=True)
dataSet.fillna(method="ffill",inplace=True)
np.where(np.isnan(dataSet))

(array([], dtype=int64), array([], dtype=int64))

In [7]:
#dataSet.head()
from sklearn.preprocessing import LabelEncoder
target=LabelEncoder().fit_transform(target)

In [8]:
    #train and test data set spilitting
    X=dataSet
    y=target
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [9]:
from xgboost import XGBClassifier
classifier = XGBClassifier()
classifier.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [10]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)

In [11]:
# Applying k-Fold Cross Validation
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
print("accuracy{:.3f}".format(accuracies.mean()))
#accuracies.std()

accuracy0.670


In [12]:
# Making the Confusion Matrix
a=confusion_matrix(y_test, y_pred)
a

array([[24,  0,  4],
       [ 0,  3,  9],
       [ 8,  3, 39]])